## Tracking the International Space Station with Dask

In this notebook, we will use two APIs: [Google Maps Geocoder](https://developers.google.com/maps/documentation/geocoding/) and the [open notify API for ISS location](http://api.open-notify.org/). We will use them to track the ISS location and next pass time in relation to a list of cities.

To help build our graphs and intelligently parallelize data, we will use [Dask](http://dask.pydata.org/en/latest/), specifically [Dask delayed](http://dask.pydata.org/en/latest/delayed.html).

In [1]:
import requests
import logging
import sys
import numpy as np
from datetime import datetime
from math import radians
from dask import delayed
from operator import itemgetter
from sklearn.neighbors import DistanceMetric

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

### First, we need to get lat and long pairs from a list of cities

In [3]:
def get_lat_long(address):
    resp = requests.get(
        'https://maps.googleapis.com/maps/api/geocode/json',
        params={'address': address}
    )
    if resp.status_code != 200:
        print('There was a problem with your request!')
        print(resp.content)
        return
    results = resp.json().get('results')
    
    if not results:
        return None
    else:
        data= results[0]
        return {
            'name': data.get('formatted_address'),
            'lat': data.get('geometry').get('location').get('lat'),
            'long': data.get('geometry').get('location').get('lng'),
        }

In [4]:
get_lat_long('Berlin, Germany')

{'lat': 52.52000659999999, 'long': 13.404954, 'name': 'Berlin, Germany'}

In [5]:
locations = []
for city in ['Seattle, Washington', 'Miami, Florida', 
             'Berlin, Germany', 'Singapore', 'Wellington, New Zealand',
             'Beirut, Lebanon', 'Beijing, China', 'Nairobi, Kenya',
             'Cape Town, South Africa', 'Buenos Aires, Argentina']:
    locations.append(get_lat_long(city))

In [6]:
locations

[{'lat': 47.6062095, 'long': -122.3320708, 'name': 'Seattle, WA, USA'},
 None,
 None,
 {'lat': 1.352083, 'long': 103.819836, 'name': 'Singapore'},
 None,
 {'lat': 33.8937913, 'long': 35.5017767, 'name': 'Beirut, Lebanon'},
 None,
 {'lat': -1.2920659, 'long': 36.8219462, 'name': 'Nairobi, Kenya'},
 None,
 None]

### Now we can define the functions we will use to get the ISS data and compare location and next pass times amongst cities 

In [7]:
def get_spaceship_location():
    resp = requests.get('http://api.open-notify.org/iss-now.json')
    location = resp.json()['iss_position']
    return {'lat': float(location.get('latitude')),
            'long': float(location.get('longitude'))}

In [8]:
def great_circle_dist(lon1, lat1, lon2, lat2):
    "Found on SO: http://stackoverflow.com/a/41858332/380442"
    dist = DistanceMetric.get_metric('haversine')
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    X = [[lat1, lon1], [lat2, lon2]]
    kms = 6367
    return (kms * dist.pairwise(X)).max()

In [9]:
def iss_dist_from_loc(issloc, loc):
    distance = great_circle_dist(issloc.get('long'), 
                                 issloc.get('lat'), 
                                 loc.get('long'), loc.get('lat'))
    logging.info('ISS is ~%dkm from %s', int(distance), loc.get('name'))
    return distance

In [15]:
def iss_pass_near_loc(loc):
    resp = requests.get('http://api.open-notify.org/iss-pass.json',
                        params={'lat': loc.get('lat'), 
                                'lon': loc.get('long')})
    data = resp.json().get('response')[0]
    td = datetime.fromtimestamp(data.get('risetime')) - datetime.now()
    m, s = divmod(int(td.total_seconds()), 60)
    h, m = divmod(m, 60)
    logging.info('ISS will pass near %s in %02d:%02d:%02d',loc.get('name'), h, m, s)
    return td.total_seconds()

In [16]:
iss_dist_from_loc(get_spaceship_location(), locations[5])

INFO:requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): api.open-notify.org
INFO:root:ISS is ~6071km from Beirut, Lebanon


6071.09615725935

In [18]:
iss_pass_near_loc(locations[7])

INFO:requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): api.open-notify.org
INFO:root:ISS will pass near Nairobi, Kenya in 00:01:17


77.885109

### Let's create a delayed pipeline

In [21]:
output = []

for loc in locations:
    issloc = delayed(get_spaceship_location)()
    dist = delayed(iss_dist_from_loc)(issloc, loc)
    output.append((loc.get('name'), dist))

closest = delayed(lambda x: sorted(x, key=itemgetter(1))[0])(output)

AttributeError: 'NoneType' object has no attribute 'get'

In [20]:
closest

NameError: name 'closest' is not defined

### Let's see our DAG!

In [33]:
closest.visualize()

NameError: name 'closest' is not defined

### Remember: it is lazy, so let's start it with `compute()`

In [22]:
closest.compute()

NameError: name 'closest' is not defined

### Exercise: which city will it fly over next?

### Extra: add your city and compare!

In [ ]:
%load ../solutions/dask.py
